### Load dataset

In [1]:
import pandas as pd

RANDOM_STATE = 42

FILE_PATH = "../Data/patient_genes_literature.csv" # Can be replaced with desired variant for different feature sets
variant = 'logistic_regression'
df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CACNA2D2  977 non-null    int64 
 1   ESR1      977 non-null    int64 
 2   AGR2      977 non-null    int64 
 3   GATA3     977 non-null    int64 
 4   SLC16A6   977 non-null    int64 
 5   TBC1D9    977 non-null    int64 
 6   INPP4B    977 non-null    int64 
 7   LDHB      977 non-null    int64 
 8   MLPH      977 non-null    int64 
 9   TSPAN1    977 non-null    int64 
 10  STBD1     977 non-null    int64 
 11  STARD3    977 non-null    int64 
 12  RARA      977 non-null    int64 
 13  MCCC2     977 non-null    int64 
 14  PSAT1     977 non-null    int64 
 15  MFGE8     977 non-null    int64 
 16  ANXA9     977 non-null    int64 
 17  PPP1R14C  977 non-null    int64 
 18  SLC44A4   977 non-null    int64 
 19  tnbc      977 non-null    bool  
 20  case_id   977 non-null    object
dtypes: bool(1), int6

### Imports

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

%run "DataHelpers.ipynb"

### Dataset split: training and test data

In [3]:
X, y, X_train, X_test, y_train, y_test, test_case_id = split_data(df, "tnbc", "case_id")

X_train.shape=(781, 19)
X_test.shape=(196, 19)
y_train.shape=(781,)
y_test.shape=(196,)


### Logistic Regression

In [4]:
# Create model
# Bumping max_iter to a higer number than the default 100, resolves the following warning
#       ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000000)

# Train the model
model.fit(X_train, y_train)

# Model predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

# Save it in a dataframe, to CSV
predictions = pd.DataFrame({
    "case_id": test_case_id,
    "y_test": y_test,
    "y_pred": y_pred,
    "y_prob": y_prob
})
predictions.to_csv(f"../Data/model_output_{variant}.csv", index=False)

# Evaluate model
print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.94


## Model cross validation

In [5]:
metrics = get_cross_validation_metrics(model, X, y, 5)
test_metrics = get_metrics(y_test, y_pred, y_prob)
test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
test = pd.DataFrame([test_metrics])
test.set_index("fold", inplace=True)

print_validated_model_accuracy(model, metrics)

# Prepend test_metrics to metrics dataframe, export and display
metrics = pd.concat([test, metrics])
metrics.to_csv(f"../Data/model_metrics_{variant}.csv", index=False)
metrics

Model validation for LogisticRegression:
[0.9336734693877551, 0.9438775510204082, 0.9538461538461539, 0.9435897435897436, 0.9435897435897436]

Mean accuracy: 0.9437



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.938776,1.000000,0.657143,0.793103,0.977633,23,161,12,0
1,0.933673,0.869565,0.666667,0.754717,0.958030,20,163,10,3
2,0.943878,0.869565,0.714286,0.784314,0.964815,20,165,8,3
3,0.953846,0.826087,0.791667,0.808511,0.959050,19,167,5,4
4,0.943590,0.869565,0.714286,0.784314,0.964105,20,164,8,3
5,0.943590,0.869565,0.714286,0.784314,0.952477,20,164,8,3
